### Importing the dataset

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
wines = pd.read_csv("Dataset/last/XWines_Slim_1K_wines.csv", encoding="utf-8")
ratings = pd.read_csv("Dataset/last/XWines_Slim_150K_ratings.csv", low_memory=False)

In [3]:
wines.head(2)

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."


In [4]:
ratings.head(5)

,RatingID,UserID,WineID,Vintage,Rating,Date
0,143,1356810,103471,1950,4.5,2021-11-02 20:52:59
1,199,1173759,111415,1951,5.0,2015-08-20 17:46:26
2,348,1164877,111395,1952,5.0,2020-11-13 05:40:26
3,374,1207665,111433,1953,5.0,2017-05-05 06:44:13
4,834,1075841,111431,1955,5.0,2016-09-14 20:18:38


### Preprocessing

#### Joining the datasets

In [5]:
wines_ratings = pd.merge(wines, ratings, on="WineID")

#### Selecting the variables

In [6]:
wines_ratings.columns

Index(['WineID', 'WineName', 'Type', 'Elaborate', 'Grapes', 'Harmonize', 'ABV',
       'Body', 'Acidity', 'Code', 'Country', 'RegionID', 'RegionName',
       'WineryID', 'WineryName', 'Website', 'Vintages', 'RatingID', 'UserID',
       'Vintage', 'Rating', 'Date'],
      dtype='object')

In [7]:
wines_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   WineID      150000 non-null  int64  
 1   WineName    150000 non-null  object 
 2   Type        150000 non-null  object 
 3   Elaborate   150000 non-null  object 
 4   Grapes      150000 non-null  object 
 5   Harmonize   150000 non-null  object 
 6   ABV         150000 non-null  float64
 7   Body        150000 non-null  object 
 8   Acidity     150000 non-null  object 
 9   Code        150000 non-null  object 
 10  Country     150000 non-null  object 
 11  RegionID    150000 non-null  int64  
 12  RegionName  150000 non-null  object 
 13  WineryID    150000 non-null  int64  
 14  WineryName  150000 non-null  object 
 15  Website     139935 non-null  object 
 16  Vintages    150000 non-null  object 
 17  RatingID    150000 non-null  int64  
 18  UserID      150000 non-null  int64  
 19  Vi

In [8]:
wines_ratings = wines_ratings[
    [
        "RatingID",
        "UserID",
        "Rating",
        "WineName",
        "Vintage",
        "Type",
        "Elaborate",
        "Grapes",
        "Harmonize",
        "ABV",
        "Body",
        "Acidity",
        "Country",
        "RegionName",
        "WineryName",
    ]
]

#### Replacing special characters

In [9]:
wines_ratings.head(3)

,RatingID,UserID,Rating,WineName,Vintage,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Country,RegionName,WineryName
0,326545,1756594,4.0,Espumante Moscatel,1999,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,Brazil,Serra Gaúcha,Casa Perini
1,1314107,1219305,2.5,Espumante Moscatel,2007,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,Brazil,Serra Gaúcha,Casa Perini
2,1446366,2047929,3.5,Espumante Moscatel,2008,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,Brazil,Serra Gaúcha,Casa Perini


In [10]:
wines_ratings['Grapes'].head()

0    ['Muscat/Moscato']
1    ['Muscat/Moscato']
2    ['Muscat/Moscato']
3    ['Muscat/Moscato']
4    ['Muscat/Moscato']
Name: Grapes, dtype: object

In [11]:
wines_ratings['Grapes'] = wines_ratings['Grapes'].str.replace("[", " ").str.replace("]", " ").str.replace("'", "").str.strip()
wines_ratings['Grapes'].head()

0    Muscat/Moscato
1    Muscat/Moscato
2    Muscat/Moscato
3    Muscat/Moscato
4    Muscat/Moscato
Name: Grapes, dtype: object

In [12]:
wines_ratings['Body'] = wines_ratings['Body'].str.replace('-', '')
wines_ratings['Body'].head()

0    Mediumbodied
1    Mediumbodied
2    Mediumbodied
3    Mediumbodied
4    Mediumbodied
Name: Body, dtype: object

In [13]:
wines_ratings["Harmonize"] = wines_ratings["Harmonize"].str.replace("[", " ").str.replace("]", " ").str.replace("'", "").str.strip()
wines_ratings["Harmonize"].head()

0    Pork, Rich Fish, Shellfish
1    Pork, Rich Fish, Shellfish
2    Pork, Rich Fish, Shellfish
3    Pork, Rich Fish, Shellfish
4    Pork, Rich Fish, Shellfish
Name: Harmonize, dtype: object

In [14]:
wines_ratings.head(3)

,RatingID,UserID,Rating,WineName,Vintage,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Country,RegionName,WineryName
0,326545,1756594,4.0,Espumante Moscatel,1999,Sparkling,Varietal/100%,Muscat/Moscato,"Pork, Rich Fish, Shellfish",7.5,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini
1,1314107,1219305,2.5,Espumante Moscatel,2007,Sparkling,Varietal/100%,Muscat/Moscato,"Pork, Rich Fish, Shellfish",7.5,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini
2,1446366,2047929,3.5,Espumante Moscatel,2008,Sparkling,Varietal/100%,Muscat/Moscato,"Pork, Rich Fish, Shellfish",7.5,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini


In [15]:
def check_special_characters(df, column_name, special_characters_pattern):
    rows_with_special_characters = df[df[column_name].str.contains(special_characters_pattern, regex=True, na=False)]
    row_count = len(rows_with_special_characters)
    print("Number of rows with special characters in {}: {}".format(column_name, row_count))
    print(rows_with_special_characters[[column_name]].drop_duplicates().sort_values(by=column_name))

In [16]:
import re
def remove_special_characters(df, column_name, characters_to_remove):
    df[column_name] = df[column_name].str.replace(f'[{re.escape(characters_to_remove)}]', ' ', regex=True)
    return df

In [17]:
characters_to_remove = '[!"#$%°&\'()*+,-./:;<=>?@[\\]^_`{|}~]'

In [18]:
check_special_characters(wines_ratings, 'WineName', characters_to_remove)

Number of rows with special characters in WineName: 34495
                                               WineName
75903                                101 Moscato d'Asti
121723               Abednego Shiraz-Grenache-Mourvedre
88926                         Adobe Carmenère (Reserva)
122656              Australia Felix Chardonnay-Viognier
75939                                    Barbera D'Alba
...                                                 ...
121869              Wallcliffe Sauvignon Blanc-Sémillon
122101       Wilyabrup Valley Cabernet Sauvignon-Merlot
119028             Winemaker's Collection Blanc de Noir
74237    Winemaker's Collection Montepulciano d'Abruzzo
148461  Красностоп Золотовский (Krasnostop Zolotovskiy)

[158 rows x 1 columns]


In [19]:
wines_ratings = remove_special_characters(wines_ratings, 'WineName', characters_to_remove)
check_special_characters(wines_ratings, 'WineName', characters_to_remove)

Number of rows with special characters in WineName: 0
Empty DataFrame
Columns: [WineName]
Index: []


In [20]:
check_special_characters(wines_ratings, 'RegionName', characters_to_remove)

Number of rows with special characters in RegionName: 20160
                                              RegionName
148704                                     Anapa (Анапа)
71856                                     Barbera d'Alba
75402                                     Barbera d'Asti
28411                                Beaujolais-Villages
75625                                  Brachetto d'Acqui
87301                                Central Valley (CL)
64898                        Chablis 1er Cru 'Montmains'
61694                                  Chambolle-Musigny
64600                     Champagne Grand Cru 'Ambonnay'
54559                        Champagne Grand Cru 'Bouzy'
63514           Champagne Grand Cru 'Le Mesnil-sur-Oger'
29073                                Châteauneuf-du-Pape
73529                  Conegliano-Valdobbiadene Prosecco
76774   Conegliano-Valdobbiadene Prosecco Superiore Rive
49819                       Corton-Charlemagne Grand Cru
84646                       

In [21]:
wines_ratings = remove_special_characters(wines_ratings, 'RegionName', characters_to_remove)
check_special_characters(wines_ratings, 'RegionName', characters_to_remove)

Number of rows with special characters in RegionName: 0
Empty DataFrame
Columns: [RegionName]
Index: []


In [22]:
check_special_characters(wines_ratings, 'WineryName', characters_to_remove)

Number of rows with special characters in WineryName: 17792
                                        WineryName
104798                              Achaval-Ferrer
59411                Barons de Rothschild (Lafite)
61506                            Barton & Guestier
137553                      Beaulieu Vineyard (BV)
137297                        Beringer Main & Vine
...                                            ...
19832                                      Warre's
24718                    Weingut Max Ferd. Richter
21002              Weingut Nik Weis-St. Urbans-Hof
24247      Winzergenossenschaft Mayschoss-Altenahr
148461  Винодельня Ведерниковъ (Vedernikov Winery)

[88 rows x 1 columns]


In [23]:
wines_ratings = remove_special_characters(wines_ratings, 'WineryName', characters_to_remove)
check_special_characters(wines_ratings, 'WineryName', characters_to_remove)

Number of rows with special characters in WineryName: 0
Empty DataFrame
Columns: [WineryName]
Index: []


#### Null values

In [24]:
wines_ratings.isnull().sum()

RatingID      0
UserID        0
Rating        0
WineName      0
Vintage       0
Type          0
Elaborate     0
Grapes        0
Harmonize     0
ABV           0
Body          0
Acidity       0
Country       0
RegionName    0
WineryName    0
dtype: int64

#### Splitting list variables

In [25]:
wines_ratings.head(3)

,RatingID,UserID,Rating,WineName,Vintage,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Country,RegionName,WineryName
0,326545,1756594,4.0,Espumante Moscatel,1999,Sparkling,Varietal/100%,Muscat/Moscato,"Pork, Rich Fish, Shellfish",7.5,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini
1,1314107,1219305,2.5,Espumante Moscatel,2007,Sparkling,Varietal/100%,Muscat/Moscato,"Pork, Rich Fish, Shellfish",7.5,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini
2,1446366,2047929,3.5,Espumante Moscatel,2008,Sparkling,Varietal/100%,Muscat/Moscato,"Pork, Rich Fish, Shellfish",7.5,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini


In [26]:
def split_and_expand_columns(df, column_name, delimiter="/"):
    expanded_columns = df[column_name].str.split(delimiter, expand=True)
    new_columns = [f"{column_name}{i+1}" for i in range(expanded_columns.shape[1])]
    expanded_columns.columns = new_columns

    df = pd.concat([df, expanded_columns], axis=1)
    df = df.fillna("")
    df = df.drop(columns=[column_name])

    return df

In [27]:
wines_ratings['Harmonize'].head()

0    Pork, Rich Fish, Shellfish
1    Pork, Rich Fish, Shellfish
2    Pork, Rich Fish, Shellfish
3    Pork, Rich Fish, Shellfish
4    Pork, Rich Fish, Shellfish
Name: Harmonize, dtype: object

In [28]:
wines_ratings = split_and_expand_columns(wines_ratings, "Harmonize", delimiter=", ")

In [29]:
wines_ratings.iloc[:, -6:].head()

,Harmonize7,Harmonize8,Harmonize9,Harmonize10,Harmonize11,Harmonize12
0,,,,,,
1,,,,,,
2,,,,,,
3,,,,,,
4,,,,,,


In [30]:
wines_ratings = split_and_expand_columns(wines_ratings, "Grapes", delimiter=", ")

In [31]:
wines_ratings.sample(15).iloc[:, -9:]

,Grapes1,Grapes2,Grapes3,Grapes4,Grapes5,Grapes6,Grapes7,Grapes8,Grapes9
138053,Chardonnay,Pinot Noir,,,,,,,
138740,Cabernet Sauvignon,,,,,,,,
6303,Cabernet Sauvignon,,,,,,,,
101851,Malbec,,,,,,,,
117104,Cabernet Sauvignon,,,,,,,,
78869,Tempranillo,Cabernet Sauvignon,,,,,,,
47148,Cabernet Franc,Merlot,Cabernet Sauvignon,,,,,,
112877,Chardonnay,,,,,,,,
13141,Touriga Nacional,Touriga Franca,Tinta Roriz,,,,,,
22319,Riesling,,,,,,,,


In [32]:
wines_ratings['Elaborate'].value_counts()

Elaborate
Varietal/100%                        105462
Assemblage/Blend                      29477
Assemblage/Bordeaux Red Blend          6737
Assemblage/Portuguese Red Blend        3170
Assemblage/Champagne Blend             1631
Assemblage/Port Blend                  1457
Varietal/>75%                           843
Assemblage/Portuguese White Blend       408
Assemblage/Rhône Red Blend              381
Assemblage/Valpolicella Red Blend       302
Assemblage/Provence Rosé Blend           99
Assemblage/Tuscan Red Blend              30
Assemblage/Priorat Red Blend              3
Name: count, dtype: int64

In [33]:
wines_ratings = split_and_expand_columns(wines_ratings, "Elaborate")

In [34]:
wines_ratings.sample(10).iloc[:, -2:]

,Elaborate1,Elaborate2
52941,Varietal,100%
111918,Varietal,100%
14882,Varietal,100%
101970,Varietal,100%
60769,Varietal,100%
124551,Varietal,100%
148470,Varietal,100%
66815,Varietal,100%
62040,Assemblage,Champagne Blend
62450,Assemblage,Champagne Blend


In [35]:
wines_ratings['Type'].value_counts()

Type
Red             98884
White           24506
Sparkling       16988
Rosé             4714
Dessert/Port     2522
Dessert          2386
Name: count, dtype: int64

In [36]:
wines_ratings = split_and_expand_columns(wines_ratings, "Type")

In [37]:
wines_ratings.sample(10).iloc[:, -2:]

,Type1,Type2
76835,Red,
62340,Sparkling,
69436,Red,
53578,Red,
85084,Red,
100753,Red,
120743,White,
140881,Dessert,
136609,Red,
83328,Rosé,


#### Alcohol beverage outliers

In [38]:
wines_ratings['ABV'].describe()

count    150000.000000
mean         13.400817
std           1.563714
min           4.000000
25%          12.500000
50%          13.500000
75%          14.000000
max          50.000000
Name: ABV, dtype: float64

In [39]:
wines_ratings['ABV'].value_counts()

ABV
14.5    22860
12.5    21716
13.5    20672
14.0    20567
13.0    19244
        ...  
38.5        4
18.5        3
39.0        2
50.0        2
44.0        2
Name: count, Length: 76, dtype: int64

In [40]:
def transform_values(df, column_name):
    df[column_name] = np.where(df[column_name] > 15, 15, df[column_name])
    df[column_name] = np.where(df[column_name] < 8, 8, df[column_name])
    return df

In [41]:
wines_ratings = transform_values(wines_ratings, 'ABV')

In [42]:
wines_ratings['ABV'].describe()

count    150000.000000
mean         13.310263
std           1.150805
min           8.000000
25%          12.500000
50%          13.500000
75%          14.000000
max          15.000000
Name: ABV, dtype: float64

In [43]:
wines_ratings['ABV'].value_counts()

ABV
14.5    22860
12.5    21716
13.5    20672
14.0    20567
13.0    19244
12.0    10234
15.0     5837
13.7     2724
13.9     2694
12.8     2627
13.2     2175
11.5     1987
13.8     1921
8.0      1882
11.0     1720
14.8     1215
14.2     1159
14.9      945
12.3      914
12.9      865
14.1      810
9.5       804
13.1      752
13.3      632
14.4      473
10.5      436
10.0      348
12.2      326
14.6      267
12.4      200
14.3      196
9.0       178
13.4      148
14.7      145
9.8       116
13.6       55
11.9       37
11.8       32
9.3        18
11.4       18
12.7       12
11.2       10
12.6        8
8.5         8
12.1        8
10.2        5
Name: count, dtype: int64

### Exporting

In [44]:
wines_ratings.to_csv('Dataset/XWines_with_Ratings.csv', index=False)